#CafChem's Drug design Agent, MoDrAg! Semantic version.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MauricioCafiero/MoDrAg/blob/main/SemanticMoDrAg/MoDrAg2_CafChem.ipynb)

## This notebook allows you to:
- Install all dependencies
- Pull the MoDrAg repo from github.
- Run the Agent

## NOTES
- GPT does not work on Colab due to dependencies that I have not been able to figure out! It *will* run on HuggingFace spaces!
- May need a HuggingFace key to access the Gemma model.

## Requirements:
- Runs well on L4 runtime; will run on T4 but may run in to memory issues with extended use.
- Needs the 07/25 runtime for deepchem to run correctly. (this is a dependency issue with the smiles tokenizer, which is used for the GPT, which, ironically, does not work on Colab due to another dependency issue!)

## Install and import libraries

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U sentence-transformers huggingface_hub==0.36.0 transformers
!pip install -q pubchempy
!pip install -q rdkit
!pip install -q chembl_webresource_client
!pip install -q rcsb-api
!pip install -q deepchem
!pip install -q dockstring
!pip install -q openbabel-wheel
!pip install -q "gliner[tokenizers]"
!pip install -q scholarly
!pip install -q httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!git clone https://github.com/MauricioCafiero/MoDrAg.git

Cloning into 'MoDrAg'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 248 (delta 16), reused 3 (delta 3), pack-reused 227 (from 1)
Receiving objects: 100% (248/248), 44.62 MiB | 15.26 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [5]:
!cp MoDrAg/SemanticMoDrAg/*.py .

cp: cannot stat 'MoDrAg/SemanticMoDrAg/*.py': No such file or directory


In [17]:
import torch
import os, re
import gradio as gr
import numpy as np
import sys

sys.path.append('MoDrAg/SemanticMoDrAg/code')

from modrag_molecule_functions import *
from modrag_property_functions import *
from finetune_gpt import *
from modrag_protein_functions import *
from modrag_task_graphs import *
from input_parsing import *
from intake_function import *


device = "cuda" if torch.cuda.is_available() else "cpu"

## GUI
- Click on the link above the Gradio window to open a full-page version of the app!

In [18]:
new_chat = chat_manager()
new_chat.start_model_tokenizer()
new_chat.start_support_models()

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/312 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

3_Dense/model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

In [19]:
with gr.Blocks() as modrag:
  top = gr.Markdown(
      """
      # Chat with MoDrAg! The MOdular DRug design AGent!
      - Use the Agent to run DD tools, chat with the underlying AI, or do a literature search.
      - Click below to see the drug design tools available.
      - Click below to see an explanation of the *modes*.
      """)
  with gr.Row():
    with gr.Accordion("Tasks available - Click to open/close..", open=False)as prot:
      gr.Markdown('''
                  - Find Uniprot IDs for a protein/gene name.
                  - report the number of bioactive molecules for a protein, organized by Chembl ID.
                  - report the SMILES and IC50 values of bioactive molecules for a particular Chembl ID.
                  - find protein sequences, report number of chains.
                  - find small molecules present in a PDB structure.
                  - find PDB IDs that match a protein.
                  - predict the IC50 value of a small molecule based on a Chembl ID.
                  - generate novel molecules based on a Chembl ID.
                  - find the name of a molecule from the SMILES string.
                  - find the SMILES string of a molecule from the name
                  - find similar or related molecules with some basic properties from a name or SMILES.
                  - calculate Lipinski properties from a name or SMILES string.
                  - find the pharmacophore-similarity between two molecules (a molecule and a reference).
                  - Find the docking score and pose coordinates for a molecules defined by a name or a SMILES string in on of the proteins below:
                  - IGF1R,JAK2,KIT,LCK,MAPK14,MAPKAPK2,MET,PTK2,PTPN1,SRC,ABL1,AKT1,AKT2,CDK2,CSF1R,EGFR,KDR,MAPK1,FGFR1,ROCK1,MAP2K1,
                  PLK1,HSD11B1,PARP1,PDE5A,PTGS2,ACHE,MAOB,CA2,GBA,HMGCR,NOS1,REN,DHFR,ESR1,ESR2,NR3C1,PGR,PPARA,PPARD,PPARG,AR,THRB,
                  ADAM17,F10,F2,BACE1,CASP3,MMP13,DPP4,ADRB1,ADRB2,DRD2,DRD3,ADORA2A,CYP2C9,CYP3A4,HSP90AA1
        ''')
    with gr.Accordion("Mode description - Click to open/close..", open=False)as mode:
      gr.Markdown('''
                  - AI: Receive an answer to your query where the AI analyses the computed data and inteprets it for you.
                  - Manual: computed data is returned to you directly with no AI interpretation.
                  - Review History: Agent uses your whole chat history in preparing your next task.
                  - Web Search: Literature search; returns 10 sources (journal articles, books, etc) ranked according to relevance.
                  - Chat: Chat with the AI about all data calculated so far, or ask any question.
        ''')

  with gr.Row():
    modes = gr.Radio(choices = ["AI", "Manual", "Review History", "Web Search", "Chat"],label="Mode Selection:",interactive=True, value = "AI", scale = 2)
    voice_choice = gr.Radio(choices = ['On', 'Off'],label="Audio Voice Response?", interactive=True, value='Off', scale = 2)

  chatbot = gr.Chatbot()


  msg = gr.Textbox(label="Type your messages here and hit enter.", scale = 2)
  with gr.Row():
    chat_btn = gr.Button(value = "Send", scale = 2)
    clear = gr.ClearButton([msg, chatbot], scale = 2)
    hard_clear = gr.ClearButton([msg, chatbot], scale = 2, value = "Hard Clear")

  image_holder = gr.Image()

  chat_btn.click(new_chat.chat, [msg, modes], [msg, chatbot, image_holder])
  msg.submit(new_chat.chat, [msg, modes], [msg, chatbot, image_holder])
  clear.click(new_chat.reset_chat)
  hard_clear.click(new_chat.hard_reset_chat)

modrag.launch(debug=True)

/tmp/ipython-input-19-2976786897.py:43: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://aecf17f96b94c87e0b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Chosen tool is: lipinski_node for query: Find the Lipinski properties for paracetamol
Second choice is: pharmfeature_node
Third choice is: smiles_node
Initial SMILES matches: []
Modified SMILES matches: []
molecules: 1
molecules: paracetamol
smiles tool
Retrieved SMILES for 1 molecules.
Initial SMILES matches: []
Modified SMILES matches: []
lipinski tool
Properties of SMILES: CC(=O)NC1=CC=C(C=C1)O: QED: 0.595
Molecular Weight: 151.165, LogP: 1.351
Hydrogen bond acceptors: 2, Hydrogen bond donors: 2
Polar Surface Area: 49.330, Rotatable Bonds: 1
Aromatic Rings: 1, Undesireable moieties: 1

Initial SMILES matches: ['Parac', 'Polar', 'polar.']
Modified SMILES matches: []
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://aecf17f96b94c87e0b.gradio.live
